In [3]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
import numpy as np
from sklearn.preprocessing import StandardScaler
import pubchempy as pcp


import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Import Data

In [ ]:
cek = pd.read_csv("Data PKM Alzheimer/string_interactions_short.tsv default node.csv")
cek

In [ ]:
def eigen(i):
    df=pd.read_csv('Data PKM Alzheimer/'+i+'.csv')
    df_name=df['name']
    df=df[['BetweennessCentrality','ClosenessCentrality','ClusteringCoefficient','Degree','Radiality','Stress','TopologicalCoefficient']]#
    scaler = StandardScaler()
    df_scaled=(scaler.fit_transform(df))
    print(df_scaled)
    n_samples = df_scaled.shape[0]
    print(n_samples)
    pca = PCA()
    X_transformed = pca.fit_transform(df_scaled)
    print(X_transformed)
    X_centered = df_scaled - np.mean(df_scaled, axis=0)
    cov_matrix = np.dot(X_centered.T, X_centered) / n_samples
    eigenvalues = pca.explained_variance_
    print('clus ',i)
    print('var ratio:',pca.explained_variance_ratio_[0])
    print('eigen value:',eigenvalues[0])
    print('eigen vector:',pca.components_[0])
    print('_______________________________________________')

    df_scaled=pd.DataFrame(df_scaled,columns=['BetweennessCentrality','ClosenessCentrality','ClusteringCoefficient','Degree','Radiality','Stress','TopologicalCoefficient'])
    df_scaled['Overall']=0.0
    for k in range(0,n_samples):
        temp=0
        for j in range(0,len(pca.components_[0])):
            temp=temp+pca.components_[0][j]*df_scaled[df_scaled.columns[j]][k]
        df_scaled['Overall'][k]=temp
        print('overall ',temp)
    pd.concat([df_name, df_scaled], axis=1, sort=False).to_excel('Data PKM Alzheimer/centrality_'+str(i)+'.xlsx')

In [ ]:
eigen('string_interactions_short.tsv default node')

In [ ]:
data=pd.read_excel('Data PKM Alzheimer/centrality_string_interactions_short.tsv default node.xlsx')
data

In [ ]:
x=data.iloc[:,1]
x

In [ ]:
y=data.iloc[:,9]
y

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8,20))
plt.barh(x, y, color='lightcoral')

plt.ylabel('Skor', size=14)
plt.xticks(size=10)
plt.yticks(size=10)

plt.show()

In [3]:
Pakai = data[data['Overall'] >= 0].sort_values(by='Overall',ascending = False)
# Pakai.to_excel('Data PKM Alzheimer/Centrality_Overall_More_Than_Zero.xlsx')
Pakai

NameError: name 'data' is not defined

In [4]:
Pakai = Pakai.reset_index(drop = True).drop(columns = 'Unnamed: 0')

NameError: name 'Pakai' is not defined

In [11]:
Nama = Pakai['name']
Nama

0          TNF
1         SNCA
2        PSEN2
3         IL1B
4        CASP3
5         RHOA
6         CDK5
7        VEGFA
8       CACUL1
9       PAXIP1
10       GSK3B
11         CLU
12      NOTCH1
13         VCP
14         IDE
15        IL10
16        CAV1
17         HTT
18       SIRT1
19        CTSB
20        ACHE
21        NOS3
22       TREM2
23       CASP8
24        BIN1
25    HLA-DRB1
26      HSPA1A
27         MME
28        SOD2
29      SQSTM1
30       ABCA1
31         SST
32          CP
33       UCHL1
34       PARK2
35       HSPA9
36      ADAM10
37      TGFBR2
38       NTRK1
39        BCHE
40       NLRP3
41       APBA1
42        PPIF
43       HDAC2
44       CD2AP
45       PRKCD
46         SHH
47        PLAU
48        NEFL
49      UBQLN1
50         NPY
51       HLA-A
52      PTPN22
53       CSF1R
54       NCSTN
55       MAP1B
56         C4B
57         TBP
58         FAS
59       ADRB2
60        CHAT
61       MTHFR
62     C9orf72
63        AGER
Name: name, dtype: object

In [12]:
smiles_dict = {}  # Empty dictionary to store key-value pairs
for i in Nama :
    results = pcp.get_compounds(i, 'name')
    for compound in results:
        Smiles = compound.isomeric_smiles
        smiles_dict[i] = Smiles

In [16]:
keys = smiles_dict.keys()

In [ ]:
# results = pcp.get_compounds('TNF', 'name')
# for compound in results:
#     print(compound.isomeric_smiles)
#     print(compound.iupac_name)
#     print(compound.synonyms)